### 🎯 Goal

We’ll use your webcam to track a colored object (for example, a blue or red ball) in real-time using OpenCV.
This teaches how to:

* Capture live video frames
* Filter specific colors (in HSV color space)
* Track and visualize movement with contours

##### Import and Setup Webcam

In [6]:
import cv2
import numpy as np

In [7]:
# Start webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Error: Could not open webcam!')
    exit()

##### Define Color Range for Tracking

In [8]:
lower_blue = np.array([90, 100, 100])
upper_blue = np.array([130, 255, 255])

##### Process Frames in Real-Time

In [9]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create mask for blue color
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Clean the mask using morphological operations
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)

    # Find contours on the mask
    contours = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500: # ignore very small areas (noise)
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, 'Tracking Object', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    # Display results
    cv2.imshow('Frame', frame)
    cv2.imshow('Mask', mask)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()